In [7]:
from typing import List, Tuple  # for type hints

import numpy as np  # for manipulating arrays
import pandas as pd  # for manipulating data in dataframes
import pickle  # for saving the embeddings cache
import plotly.express as px  # for plots
import random  # for generating run IDs
from sklearn.model_selection import train_test_split  # for splitting train & test data
import torch  # for matrix optimization

from openai.embeddings_utils import get_embedding, cosine_similarity  # for embeddings

reaction_type = {"<RX_6>": 0, "<RX_2>": 1, "<RX_1>": 2, "<RX_3>": 3, "<RX_7>": 4, "<RX_9>": 5, "<RX_5>": 6, "<RX_10>": 7, "<RX_4>": 8, "<RX_8>": 9}

# input parameters
embedding_cache_path = "data/uspto50_embedding_cache.pkl"  # embeddings will be saved/loaded here
default_embedding_engine = "babbage-similarity"  # text-embedding-ada-002 is recommended
num_pairs_to_embed = 1000  # 1000 is arbitrary
local_dataset_path = "data/uspto_50.csv"  # download from: https://nlp.stanford.edu/projects/snli/
train_dataset_path = "data/uspto_50_validpair.csv"
test_dataset_path = "data/uspto_50_testpair.csv"


def process_input_data(df: pd.DataFrame) -> pd.DataFrame:
    # you can customize this to preprocess your own dataset
    # output should be a dataframe with 3 columns: text_1, text_2, label (1 for similar, -1 for dissimilar)
    df_temp = {
        "text_1": [],
        "text_2": [],
        "label": [],
    }
    df_len = len(df)
    for i in df_len:
        j = i + 1
        while j < df_len:
            if df["set"][i] == "train" and df["set"][j] == "train":
                if i < j:
                    df_temp["text_1"].append(df["products_mol"][i])
                    df_temp["text_2"].append(df["products_mol"][j])
                    if df["reaction_type"][i] == df["reaction_type"][j]:
                        df_temp["label"].append(1)
                    else:
                        df_temp["label"].append(-1)
            if df["set"][i] == "test" and df["set"][j] == "test":
                if i < j:
                    df_temp["text_1"].append(df["products_mol"][i])
                    df_temp["text_2"].append(df["products_mol"][j])
                    if df["reaction_type"][i] == df["reaction_type"][j]:
                        df_temp["label"].append(1)
                    else:
                        df_temp["label"].append(-1)
        if i % 1000 == 0:
            print(i)
    # df = df.head(num_pairs_to_embed)
    return df_temp

# load data
df_train = pd.read_csv(train_dataset_path)

def get_embedding_with_cache(
    text: str,
    engine: str = default_embedding_engine,
    embedding_cache: dict = df_train,
    embedding_cache_path: str = embedding_cache_path,
) -> list:
    print(f"Getting embedding for {text}")
    if (text, engine) not in embedding_cache.keys():
        # if not in cache, call API to get embedding
        embedding_cache[(text, engine)] = get_embedding(text, engine)
        # save embeddings cache to disk after each update
        with open(embedding_cache_path, "wb") as embedding_cache_file:
            pickle.dump(embedding_cache, embedding_cache_file)
    return embedding_cache[(text, engine)]


a = get_embedding("Two men with heads down signing a paper.", default_embedding_engine)
a
# # create column of embeddings
# for column in ["text_1", "text_2"]:
#     df_train[f"{column}_embedding"] = df_train[column].apply(get_embedding_with_cache)

# # create column of cosine similarity between embeddings
# df_train["cosine_similarity"] = df_train.apply(
#     lambda row: cosine_similarity(row["text_1_embedding"], row["text_2_embedding"]),
#     axis=1,
# )
# process input data
# df = process_input_data(df)  # this demonstrates training data containing only positives
# df = pd.DataFrame(df)
# view data
# df.head()
# df.to_csv("data/uspto_50_type_pair.csv", index=False)

RetryError: RetryError[<Future at 0x23cb84c6410 state=finished raised AuthenticationError>]